# Import Library 

In [1]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from function_util import *
from SMP1 import SMP1
from FIS2 import FIS2
from datetime import datetime

In [2]:
gdf_utm = pd.read_pickle('gdf_utm.pkl')
edges_utm = pd.read_pickle('edges_utm.pkl')
nodes_utm = pd.read_pickle('nodes_utm.pkl')
curr_edge = pd.read_pickle('current_edge.pkl')


In [3]:
# for debugging purposes start at iteration point 14 
point_index = 13
# this is for debugging purposes, true 
prev_loc = gdf_utm.iloc[[point_index - 1]]
last_matched = point_matching(prev_loc, curr_edge.iloc[0])

# current location 
curr_loc = gdf_utm.iloc[[point_index]]

# find longitude and latitude for last matched data
last_matched['lon_lat'] = last_matched.to_crs({'init': 'epsg:4326'})

res_13 = SMP1(curr_loc, curr_edge, prev_loc, last_matched, nodes_utm, edges_utm, gdf_utm)

C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [4]:
edges_utm

osmid   lanes     ref  \
u           v           key                                                     
28249736    279006817   0                            32922734       3  L 1198   
            5474949357  0                           569246377       3  L 1198   
            279007200   0                            25595739       1     NaN   
279006817   28249736    0                            32922734       3  L 1198   
            522063015   0    [26244402, 569260268, 231768647]  [2, 3]  L 1198   
...                                                       ...     ...     ...   
10011084744 10011084745 0                          1093116033     NaN     NaN   
10011084746 10011084745 0                          1093116034     NaN     NaN   
10967832974 10967832973 0                          1180976329     NaN     NaN   
10967832976 10967832975 0                          1180976330     NaN     NaN   
10967842019 10967842023 0                          1180976942     NaN     NaN   

                                                   name    highway  maxspeed  \
u           v           key                                                    
28249736    279006817   0    Hedelfinger Filderauffahrt  secondary        50   
            5474949357  0                  Bockelstraße  secondary        50   
            279007200   0                  Bockelstraße   tertiary        50   
279006817   28249736    0    Hedelfinger Filderauffahrt  secondary        50   
            522063015   0    Hedelfinger Filderauffahrt  secondary  [60, 50]   
...                                                 ...        ...       ...   
10011084744 10011084745 0                           NaN      steps       NaN   
10011084746 10011084745 0                           NaN      steps       NaN   
10967832974 10967832973 0                           NaN    service       NaN   
10967832976 10967832975 0                           NaN    service       NaN   
10967842019 10967842023 0                           NaN    service       NaN   

                             oneway       reversed  length  \
u           v           key                                  
28249736    279006817   0     False           True   8.219   
            5474949357  0     False          False   6.057   
            279007200   0      True          False  31.858   
279006817   28249736    0     False          False   8.219   
            522063015   0     False  [False, True]  73.784   
...                             ...            ...     ...   
10011084744 10011084745 0     False           True   6.863   
10011084746 10011084745 0     False           True   9.018   
10967832974 10967832973 0     False           True  20.951   
10967832976 10967832975 0     False           True  16.971   
10967842019 10967842023 0     False          False  28.105   

                                                                      geometry  \
u           v           key                                                      
28249736    279006817   0    LINESTRING (516726.631 5398716.229, 516721.392...   
            5474949357  0    LINESTRING (516726.631 5398716.229, 516728.559...   
            279007200   0    LINESTRING (516726.631 5398716.229, 516738.592...   
279006817   28249736    0    LINESTRING (516721.392 5398722.572, 516726.631...   
            522063015   0    LINESTRING (516721.392 5398722.572, 516713.337...   
...                                                                        ...   
10011084744 10011084745 0    LINESTRING (517242.687 5400711.583, 517245.976...   
10011084746 10011084745 0    LINESTRING (517237.588 5400708.921, 517245.976...   
10967832974 10967832973 0    LINESTRING (518681.028 5400444.184, 518660.298...   
10967832976 10967832975 0    LINESTRING (518615.307 5400532.160, 518598.813...   
10967842019 10967842023 0    LINESTRING (518712.403 5400446.845, 518713.675...   

                            width   service bridge junction access tunnel  \
u         

In [5]:
# for debugging purposes start at iteration point 14 
point_index = 14
# this is for debugging purposes, true 
prev_loc = gdf_utm.iloc[[point_index - 1]]
last_matched = point_matching(prev_loc, curr_edge.iloc[0])

# current location 
curr_loc = gdf_utm.iloc[[point_index]]

# find longitude and latitude for last matched data
last_matched['lon_lat'] = last_matched.to_crs({'init': 'epsg:4326'})

res_14 = SMP1(curr_loc, curr_edge, prev_loc, last_matched, nodes_utm, edges_utm, gdf_utm)

C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [6]:
[res_13, res_14]

[10.203497145276039, 39.999999999343814]

In [ ]:
# Debugging purposes
[19.881427029655967, 19.999900237642525]

In [8]:
def SMP1(curr_loc, curr_edge, prev_loc, last_matched, nodes_utm, edges_utm, gdf_utm):
    
    start_node = nodes_utm.loc[curr_edge.index[0][0]]
    end_node = nodes_utm.loc[curr_edge.index[0][1]]

    # finding angle alpha, which is the angle between previous matched point and current postion 
    # find pos_bearing1, bearing between last matched point and current point
    # convert two points into tuple for function input purposes
    last_matched_tuple = (last_matched['lon_lat'].x.iloc[0], last_matched['lon_lat'].y.iloc[0])
    cur_loc_tuple = curr_loc['lon_lat'].iloc[0].x, curr_loc['lon_lat'].iloc[0].y
    pos_bearing1 = get_bearing(last_matched_tuple, cur_loc_tuple)
    # convert to 0, 2pi 
    pos_bearing1 = conv_angle(pos_bearing1)

    # find bearing of the edge 
    # convert lat lon into tupple coordinate 
    start_node_tuple = (start_node.lon, start_node.lat)
    end_node_tuple = (end_node.lon, end_node.lat)
    # bearing between start to end
    start_bearing = get_bearing(start_node_tuple, end_node_tuple)
    # convert to 0 ,2*pi
    start_bearing = conv_angle(start_bearing)
    alpha = abs(pos_bearing1 - start_bearing)

    # find beta, angle between end node and current position 
    # bearing from end to start 
    end_bearing = get_bearing(end_node_tuple, start_node_tuple)
    end_bearing = conv_angle(end_bearing)
    #pos_bearing2 beating between end node and current position 
    pos_bearing2 = get_bearing(end_node_tuple, cur_loc_tuple)
    pos_bearing2 = conv_angle(pos_bearing2)
    # beta equal to abs(end_bearing minus pos_bearing2)
    beta = abs(end_bearing - pos_bearing2)


    # Heading Increment  
    HI = curr_loc['GPS Bearing'].iloc[0] - prev_loc['GPS Bearing'].iloc[0]

    # distance travelled from last position fix to the end nodes
    d = end_node['geometry'].distance(last_matched['geometry']).iloc[0]

    # distance travelled since last position fix 
    t = curr_loc['time'].iloc[0] - prev_loc['time'].iloc[0]
    d2 = (curr_loc['speed_mps'].iloc[0] )* t.seconds

    delta_d = d - d2

    speed = curr_loc['speed_mps'].iloc[0] 
    hdop = curr_loc['GPS HDOP'].iloc[0] 

    # rearrange new data to the input of fis1  
    new_data = np.array([speed,hdop, alpha, beta, delta_d, HI, HI]).T

    res = FIS2(new_data, plot = False)
    
    return res

# Visualizing SMP 1 

In [37]:
%matplotlib tk
# This is how we  visualize edges and error bound 

err_size = 38 
# find which edges is selected at time point
# find index of the edge id
# find the last two position for IMP
poly_1 = err_polygon(gdf_utm.iloc[[13]], err_size)
poly_2 = err_polygon(gdf_utm.iloc[[14]], err_size)

# plotting edges and starting point together 
f, ax = plt.subplots()


# location for all point
#locs_utm.plot(ax=ax)
point_locs = gdf_utm['geometry'].to_frame()
point_locs.plot(ax = ax)

#err coord for location at iteration 13 and 14
poly_1.plot(ax=ax, facecolor="none")
poly_2.plot(ax=ax, facecolor="none")

# this plot all the road system 
edges_utm.plot(ax=ax)
# this plot the selected edge at time point 

# matched point plot
last_matched.plot(ax = ax, color = "Green")



# # plot closest node
# closest_node.plot(ax = ax, color = "Black")
# closest_edge.plot(ax = ax , color = "Black")
# # debuging for djiksta shortest path
# origin_node.plot(ax = ax, color = "Black")
# target_node.plot(ax = ax , color = "Black")

<Axes: >